In [89]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [90]:
import cython
%load_ext cython
print(cython.__version__)

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
3.0.0


In [91]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]

In [92]:
GAME_COLUMNS = ["chance"] + ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS] + DECK_COLUMNS

# SETUP

In [93]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [94]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = update_player_with_card(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = update_dealer_with_card(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

In [95]:
def update_player_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [96]:
def update_dealer_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [97]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

# PLAYER

In [98]:
def player_hits_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    next_game = next_game.groupby(next_game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [99]:
def player_hits_once_where_possible(game, verbose=False):
    """get updated game when player hits once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        if verbose : print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        if verbose : print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    # group and reorder
    final = final.groupby(final.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    final = order_game_columns(final)

    return final.sort_values("player_score")

# DEALER

In [100]:
def dealer_get_score_chances_for_deck(game):
    assert set(DECK_COLUMNS).issubset(set(game.columns.tolist())), "Deck Info not in columns"
    assert set(["dealer_score","dealer_aces","dealer_cards","chance"]).issubset(game.columns.tolist()), "not all dealer info in columns"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"

    # keep original deck info
    for deck_column in DECK_COLUMNS:
        game["orig_"+deck_column] = game[deck_column]

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    while len(playable) > 0:
        # add possible cards to draw
        next_game = pd.merge(playable, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

        # helper diag matrix
        diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(playable),1))

        # update drawing chance
        next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values
        
        # TODO: 0 boundary check, what if there are no cards of that type left?
        # currently works, because the chance is 0 anyways

        # update deck info
        next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

        # updating score
        next_game["dealer_score"] += next_game["drawn"]
        # updating cards
        next_game["dealer_cards"] += 1
        # updating aces
        next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

        # if necessary, using ace as 1 instead of 11
        acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
        next_game.loc[acenecessary,"dealer_score"] -= 10
        next_game.loc[acenecessary,"dealer_aces"] -= 1

        # set bust to -2
        next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

        # set blackjack to 22
        next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

        # removing drawn column
        next_game = next_game.drop("drawn", axis="columns")
        
        cols = game.columns.tolist()
        cols.remove("chance")
        next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()

        is_playable = (next_game["dealer_score"] < 17) & (next_game["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_game[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_game[is_playable]        

    grouped = final.groupby(["orig_"+col for col in DECK_COLUMNS] + ["player_score","player_cards","dealer_score","dealer_aces","dealer_cards"]).agg({'chance':'sum'}).reset_index()
    
    return grouped

In [101]:
def dealer_draws_once(game):
    """get updated game when dealer draws one card, throws error for finished hands"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to draw. hands:\n{game}"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values
    
    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    next_game = next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()

    next_game = order_game_columns(next_game)

    return next_game.sort_values("dealer_score")

In [102]:
def dealer_plays(game):
    """get updated game when dealer plays until 17 or bust"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    if len(playable) == 0:
        if verbose : print("Dealer has finished all hands")
        return game

    while len(playable) > 0:
        next_possible = dealer_draws_once(playable)

        is_playable = (next_possible["dealer_score"] < 17) & (next_possible["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_possible[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_possible[is_playable]
    
    return final

# RETURNS

In [103]:
def get_return_finished(game):
    """get the probabilities of each possible return of the current state for the specified game
    no further moves are made"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    # blackjack win
    blackjack_win = (game["player_score"] == 22) & (game["dealer_score"] != 22)
    blackjack_win_chance = game.loc[blackjack_win, "chance"].sum()

    # win
    win = (game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)
    win_chance = game.loc[win, "chance"].sum()

    # lose
    lose = (game["player_score"] < game["dealer_score"])
    lose_chance = game.loc[lose, "chance"].sum()

    # push
    push = (game["player_score"] == game["dealer_score"])
    push_chance = game.loc[push, "chance"].sum()

    expected_return = blackjack_win_chance*1.5 + win_chance*1 + push_chance*0 + lose_chance*(-1)

    return pd.Series({
        "blackjack_win_chance": blackjack_win_chance,
        "win_chance": win_chance,
        "push_chance": push_chance,
        "lose_chance": lose_chance,
        "expected_return": expected_return
    })

In [104]:
def get_return_standing(game, verbose=False):
    """get the expected return of standing (or doing nothing in case the player is already finished)"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = dealer_plays(game.copy())

    if verbose : print(game)

    return get_return_finished(game)

In [105]:
def get_return_hitting_once(game, verbose=False):
    """get the expected return of hitting exactly once, throws error for finished hands"""
    
    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return get_return_standing(game, verbose=verbose)

In [106]:
def get_return_hitting_once_where_possible(game, verbose=False):
    """get the expected return of hitting exactly once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    game = player_hits_once_where_possible(game.copy())

    return get_return_standing(game, verbose=verbose)

In [107]:
def get_return_doubling_down(game, verbose=False):
    """get the expected return of doubling down, throws error for finished hands"""

    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] == 2), "player does not have exactly 2 cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return_chances = get_return_standing(game, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [108]:
def get_return_doubling_down_where_possible(game, verbose=False):
    """get the expected return of doubling down, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    return_chances = get_return_standing(final, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [109]:
def get_return_column(game):
    """get column denoting the return for each game, assuming its finished"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    return_column = pd.Series(np.zeros(len(game)))

    return_column[(game["player_score"] == 22) & (game["dealer_score"] != 22)] = 1.5
    return_column[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)] = 1
    return_column[game["player_score"] == game["dealer_score"]] = 0
    return_column[game["player_score"] < game["dealer_score"]] = -1

    return return_column

# TESTING

In [110]:
def get_best_return(game, recursive=False):
    """get the expected return of the current state, assuming best play
    Only standing and hitting are implemented yet"""

    if not recursive:
        print(game)

    stand_return = get_return_standing(game)
    if not recursive:
        print("Stand return\n", stand_return)

    playable = game[(game["player_score"] >= 0) & (game["player_score"] != 22)].copy()

    if len(playable) == 0:
        return pd.Series({
        "stand":stand_return["expected_return"], 
        "hit":-1, 
        "best":stand_return["expected_return"],
        "option":"stand"
        })
    
    hit_game = player_hits_once(playable, keep_drawn_column=True)
    hit_game = hit_game.rename(columns={"drawn":"drawn_first"})
    hit_game_finished = dealer_plays(hit_game.copy())
    hit_game_finished["return"] = get_return_column(hit_game_finished)
    hit_game_finished["expected_return"] = hit_game_finished["chance"] * hit_game_finished["return"]
    

    hit_game_grouped = hit_game_finished.groupby("drawn_first").agg({"expected_return":"sum"}).reset_index().sort_values("drawn_first")
    if not recursive:
        print("Hit return\n", hit_game_grouped)
        print(hit_game_grouped["expected_return"].sum())

    for i in hit_game_grouped.index:
        drawn_card = hit_game_grouped.loc[i,"drawn_first"]
        

        if len(hit_game[(hit_game["player_score"] >= 0) & (hit_game["player_score"] != 22) & (hit_game["drawn_first"] == drawn_card)]) == 0 :
            continue # no need to simulate this draw, it is already busted
            
        if not recursive:
            print("sim", drawn_card)

        if hit_game[(hit_game["drawn_first"] == drawn_card)]["player_score"].min() == 21:
            if not recursive:
                print("is 21, so standing is better")
            continue # no need to simulate, hand is 21
        
        if not recursive:
            print("sim\n", hit_game[hit_game["drawn_first"] == drawn_card].copy().drop(columns=["drawn_first"]))
        hit_return = get_best_return(hit_game[hit_game["drawn_first"] == drawn_card].copy().drop(columns=["drawn_first"]), recursive=True)

        if not recursive:
            print("hitting after\n", hit_return)
        
        if hit_return["hit"] > hit_game_grouped.loc[i,"expected_return"] and not recursive:
            print("\nBetter return when hitting after", drawn_card, ":\n", hit_return, "\nvs\n", hit_game_grouped.loc[i])
        hit_game_grouped.loc[i,"expected_return"] = max(hit_return["hit"], hit_game_grouped.loc[i,"expected_return"])
    
    if not recursive:
        print("Updated hit return\n", hit_game_grouped)
        print("expected return", hit_game_grouped["expected_return"].sum())
        print("Stand return\n", stand_return)

    return_series = pd.Series({
        "stand":stand_return["expected_return"], 
        "hit":hit_game_grouped["expected_return"].sum(), 
        "best":max(stand_return["expected_return"], hit_game_grouped["expected_return"].sum()),
        "option":"stand" if stand_return["expected_return"] > hit_game_grouped["expected_return"].sum() else "hit"
        })
    return return_series

if False:
    game = get_game(None, [6,6], [3], update_deck=False)
    r = get_best_return(game)
    r

In [111]:
def get_best_return_updated(game):
    """get the expected return of the current state, assuming best play
    Only standing and hitting are implemented yet"""

    t1 = time.time()

    stand_return = get_return_standing(game)

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)

    playable = game[is_playable].copy()
    final = game.copy()

    print("calculating all possible moves...")
    while len(playable) > 0:
        next_game = player_hits_once(playable, keep_drawn_column=True)
        drawnsum = len([i for i in next_game.columns.tolist() if "drawn" in i])
        next_game = next_game.rename(columns={"drawn":"drawn_"+str(drawnsum)})
        final["drawn_"+str(drawnsum)] = np.zeros(len(final)).astype(int)

        is_playable = (next_game["player_score"] >= 0) & (next_game["player_score"] != 22) & (next_game["player_score"] != 21)
        final = pd.concat((final, next_game), axis=0, ignore_index=True)

        playable = next_game[is_playable].copy()

    print("calculated all player moves", time.time() - t1)
    print(final)

    return final

    # this is not performant, as the dealer acts on every move. Instead group by available deck cards and then calculate.

    final = dealer_plays(final)
    print("calculated dealer moves", time.time() - t1)
    print(final)
    
    return_column = get_return_column(final)
    final["return"] = return_column
    final["expected_return"] = final["chance"] * final["return"]

    drawn_columns = [i for i in final.columns.tolist() if "drawn" in i]

    for i in range(len(drawn_columns)):
        col = drawn_columns[len(drawn_columns)-i-1]
        cols = drawn_columns[:len(drawn_columns)-i-1]
        print(col, time.time() - t1)

        # get all games that can stand or hit in that column
        if len(cols) != 0:
            useable = (final[col] != 0) | ((final[col] == 0) & (final[cols[-1]] != 0) & (final["player_score"] > 0))
        else:
            useable = (final[col] != 0) | ((final[col] == 0) & (final["player_score"] > 0))
        used = final[useable].copy()
        stand_column = used[col] == 0
        used["stand"] = stand_column
        # print(used.sort_values(drawn_columns))
        grouped = used.groupby(cols + ["stand"]).agg({'chance': 'sum', "expected_return": "sum"}).reset_index()
        # print(grouped)

        if len(cols) != 0:
            best_options = grouped.groupby(cols).agg({"expected_return": "max", "chance":"mean"}).reset_index()
        else:
            return grouped.drop(columns=["chance"])
        
        # print(best_options)
        

        final = final[~useable].copy().drop(columns=[col])
        # print(final)
        retdf = pd.DataFrame({
            "chance": best_options["chance"],
            "expected_return": best_options["expected_return"]
        })
        for c in cols:
            retdf[c] = best_options[c]
        
        # print(retdf)
        final = pd.concat([final, retdf], axis=0, ignore_index=True)
        # print(final)
    
    return final


In [161]:
game = get_game(None, [6,6], [10], update_deck=True)

final = get_best_return_updated(game)
final

calculating all possible moves...
calculated all player moves 0.29463791847229004
            chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4  ...  11  drawn_1  drawn_2  drawn_3  drawn_4  drawn_5  drawn_6  drawn_7  drawn_8  drawn_9
0     1.000000e+00            12             2            0            10             1            0  24  24  24  ...  24        0        0        0        0        0        0        0        0        0
1     3.074434e-01            -2             3            0            10             1            0  24  24  24  ...  24       10        0        0        0        0        0        0        0        0
2     7.766990e-02            13             3            0            10             1            0  24  24  24  ...  23       11        0        0        0        0        0        0        0        0
3     7.766990e-02            14             3            0            10             1            0  23  

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,...,11,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9
0,1.000000e+00,12,2,0,10,1,0,24,24,24,...,24,0,0,0,0,0,0,0,0,0
1,3.074434e-01,-2,3,0,10,1,0,24,24,24,...,24,10,0,0,0,0,0,0,0,0
2,7.766990e-02,13,3,0,10,1,0,24,24,24,...,23,11,0,0,0,0,0,0,0,0
3,7.766990e-02,14,3,0,10,1,0,23,24,24,...,24,2,0,0,0,0,0,0,0,0
4,7.766990e-02,15,3,0,10,1,0,24,23,24,...,24,3,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,3.117015e-11,-2,11,0,10,1,0,24,24,24,...,16,11,11,11,11,11,11,11,11,7
2557,3.117015e-11,-2,11,0,10,1,0,24,24,24,...,16,11,11,11,11,11,11,11,11,8
2558,3.117015e-11,-2,11,0,10,1,0,24,24,24,...,16,11,11,11,11,11,11,11,11,9
2559,1.233818e-10,-2,11,0,10,1,0,24,24,24,...,16,11,11,11,11,11,11,11,11,10


In [162]:
# assertions to check that everything is equal
final_grouped_same = final.groupby(DECK_COLUMNS + ["player_score","player_cards","dealer_score","dealer_aces","dealer_cards"]).agg({"chance":"sum"}).reset_index()
final_grouped_same

,2,3,4,5,6,7,8,9,10,11,player_score,player_cards,dealer_score,dealer_aces,dealer_cards,chance
0,19,24,24,24,22,24,24,24,95,24,-2,7,10,0,1,0.000002
1,20,23,24,24,22,24,24,24,95,24,-2,7,10,0,1,0.000002
2,20,24,23,24,22,24,24,24,95,24,-2,7,10,0,1,0.000002
3,20,24,24,23,22,24,24,24,95,24,-2,7,10,0,1,0.000002
4,20,24,24,24,21,24,24,24,95,24,-2,7,10,0,1,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,24,24,24,24,22,24,24,24,95,20,16,6,10,0,1,0.000029
516,24,24,24,24,22,24,24,24,95,21,15,5,10,0,1,0.000416
517,24,24,24,24,22,24,24,24,95,22,14,4,10,0,1,0.005800
518,24,24,24,24,22,24,24,24,95,23,13,3,10,0,1,0.077670


In [168]:
# could also include player score and then just check for win/push/lose
dealer_scores = dealer_get_score_chances_for_deck(final_grouped_same)
dealer_scores["return"] = get_return_column(dealer_scores)
dealer_scores["expected_return"] = dealer_scores["chance"] * dealer_scores["return"]
dealer_scores
dealer_scores.loc[np.all(dp.loc[:,"orig_2":"orig_11"].values == [24,24,24,24,22,24,24,24,95,24], axis=1)]


,orig_2,orig_3,orig_4,orig_5,orig_6,orig_7,orig_8,orig_9,orig_10,orig_11,player_score,player_cards,dealer_score,dealer_aces,dealer_cards,chance,return,expected_return
18165,24,24,24,24,22,24,24,24,95,24,12,2,-1,0,3,1.755979e-01,1.0,1.755979e-01
18166,24,24,24,24,22,24,24,24,95,24,12,2,-1,0,4,3.249197e-02,1.0,3.249197e-02
18167,24,24,24,24,22,24,24,24,95,24,12,2,-1,0,5,2.659958e-03,1.0,2.659958e-03
18168,24,24,24,24,22,24,24,24,95,24,12,2,-1,0,6,1.014066e-04,1.0,1.014066e-04
18169,24,24,24,24,22,24,24,24,95,24,12,2,-1,0,7,1.395442e-06,1.0,1.395442e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,24,24,24,24,22,24,24,24,95,24,12,2,21,0,4,4.553936e-03,-1.0,-4.553936e-03
18196,24,24,24,24,22,24,24,24,95,24,12,2,21,0,5,3.499558e-04,-1.0,-3.499558e-04
18197,24,24,24,24,22,24,24,24,95,24,12,2,21,0,6,1.296186e-05,-1.0,-1.296186e-05
18198,24,24,24,24,22,24,24,24,95,24,12,2,21,0,7,1.771989e-07,-1.0,-1.771989e-07


In [164]:
dealer_scores_grouped = dp.groupby(["orig_"+c for c in DECK_COLUMNS]+["player_score","player_cards"]).agg({"expected_return":"sum","chance":"sum"}).reset_index()
dealer_scores_grouped.loc[np.all(dealer_scores_grouped.loc[:,"orig_2":"orig_11"].values == [23,24,23,24,22,24,24,24,95,22], axis=1),:]
dealer_scores_grouped.sort_values("expected_return")

,orig_2,orig_3,orig_4,orig_5,orig_6,orig_7,orig_8,orig_9,orig_10,orig_11,player_score,player_cards,expected_return,chance
519,24,24,24,24,22,24,24,24,95,24,12,2,-0.578295,1.000000
509,24,24,24,24,22,24,24,24,94,24,-2,3,-0.307443,0.307443
253,23,24,24,24,22,24,24,24,95,24,14,3,-0.045003,0.077670
354,24,23,24,24,22,24,24,24,95,24,15,3,-0.044996,0.077670
405,24,24,23,24,22,24,24,24,95,24,16,3,-0.044990,0.077670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,23,24,24,24,22,23,24,24,95,24,21,4,0.009801,0.012104
371,24,24,23,23,22,24,24,24,95,24,21,4,0.009803,0.012104
490,24,24,24,24,22,24,23,24,95,23,21,4,0.009876,0.012104
491,24,24,24,24,22,24,23,24,95,24,20,3,0.033671,0.077670


In [186]:
# now map the origs to the expected returns
# then extract best moves ike before
final_with_er = pd.merge(final.copy().drop(columns=["chance"]), dealer_scores_grouped.loc[:, ["orig_"+s for s in DECK_COLUMNS] + ["expected_return","chance"]], left_on=DECK_COLUMNS, right_on=["orig_"+s for s in DECK_COLUMNS], how="left")
final_with_er

,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,...,orig_4,orig_5,orig_6,orig_7,orig_8,orig_9,orig_10,orig_11,expected_return,chance
0,12,2,0,10,1,0,24,24,24,24,...,24,24,22,24,24,24,95,24,-5.782948e-01,1.000000e+00
1,-2,3,0,10,1,0,24,24,24,24,...,24,24,22,24,24,24,94,24,-3.074434e-01,3.074434e-01
2,13,3,0,10,1,0,24,24,24,24,...,24,24,22,24,24,24,95,23,-4.478509e-02,7.766990e-02
3,14,3,0,10,1,0,23,24,24,24,...,24,24,22,24,24,24,95,24,-4.500325e-02,7.766990e-02
4,15,3,0,10,1,0,24,23,24,24,...,24,24,22,24,24,24,95,24,-4.499629e-02,7.766990e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,-2,11,0,10,1,0,24,24,24,24,...,24,24,22,23,24,24,95,16,-3.117015e-11,3.117015e-11
2557,-2,11,0,10,1,0,24,24,24,24,...,24,24,22,24,23,24,95,16,-3.117015e-11,3.117015e-11
2558,-2,11,0,10,1,0,24,24,24,24,...,24,24,22,24,24,23,95,16,-3.117015e-11,3.117015e-11
2559,-2,11,0,10,1,0,24,24,24,24,...,24,24,22,24,24,24,94,16,-1.233818e-10,1.233818e-10


In [185]:
# checks
final_with_er[final_with_er["expected_return"] >= final_with_er["chance"]].sort_values("player_score")

,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,...,orig_4,orig_5,orig_6,orig_7,orig_8,orig_9,orig_10,orig_11,expected_return,chance


In [120]:
###### below is no longer relevant

# OLD

In [121]:
g = get_game(None, [11,11], [11], update_deck=False)
print(g)
dealer_plays(g)

   chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0     1.0            12             2            1            11             1            1  24  24  24  24  24  24  24  24  96  24


,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,7.692308e-02,12,2,1,17,2,1,24,24,24,24,23,24,24,24,96,24
1,7.692308e-02,12,2,1,18,2,1,24,24,24,24,24,23,24,24,96,24
2,7.692308e-02,12,2,1,19,2,1,24,24,24,24,24,24,23,24,96,24
3,7.692308e-02,12,2,1,20,2,1,24,24,24,24,24,24,24,23,96,24
4,3.076923e-01,12,2,1,22,2,1,24,24,24,24,24,24,24,24,95,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,7.485829e-14,12,2,1,17,12,0,24,24,24,24,23,24,24,24,96,14
1355,1.197733e-13,12,2,1,18,12,0,23,24,24,24,23,24,24,24,96,15
1356,1.197733e-13,12,2,1,19,12,0,24,23,24,24,23,24,24,24,96,15
1357,1.197733e-13,12,2,1,20,12,0,24,24,23,24,23,24,24,24,96,15


In [122]:
get_return_standing(game)

blackjack_win_chance    0.000000
win_chance              0.210853
push_chance             0.000000
lose_chance             0.789147
expected_return        -0.578295
dtype: float64

In [123]:
np.all(r.loc[:,"drawn_1":"drawn_9"] == 11, axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
2556    False
2557    False
2558    False
2559    False
2560     True
Length: 2561, dtype: bool

In [124]:
elevens = r[np.all(r.loc[:,"drawn_1":"drawn_9"] == 11, axis=1)]
elevens

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,...,11,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9
2560,2.078010e-11,21,11,0,10,1,0,24,24,24,...,15,11,11,11,11,11,11,11,11,11


In [125]:
print("standing")
print(elevens[elevens["drawn_10"] == 0]["chance"].sum())
print(elevens[elevens["drawn_10"] == 0]["expected_return"].sum())
print("hitting")
print(elevens[elevens["drawn_10"] != 0]["chance"].sum())
print(elevens[elevens["drawn_10"] != 0]["expected_return"].sum())

standing


KeyError: 'drawn_10'

In [ ]:
game = get_game(None, [6,6], [3], update_deck=False)
r = get_best_return(game)
r

   chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0     1.0            12             2            0             3             1            0  24  24  24  24  24  24  24  24  96  24
Stand return
 blackjack_win_chance    0.00000
win_chance              0.37395
push_chance             0.00000
lose_chance             0.62605
expected_return        -0.25210
dtype: float64
Hit return
    drawn_first  expected_return
0            2        -0.019378
1            3        -0.019355
2            4        -0.019178
3            5        -0.008779
4            6         0.011624
5            7         0.031329
6            8         0.050230
7            9         0.068093
8           10        -0.307692
9           11        -0.019283
-0.23238828771219874
sim 2
sim
      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
2  0.076923      

stand      -0.2521
hit      -0.232388
best     -0.232388
option         hit
dtype: object

In [ ]:
print(pd.DataFrame({
    "standing":get_return_standing(game), 
    "hitting_once":get_return_standing(player_hits_once_where_possible(game)), 
    "hitting_twice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(game))), 
    "hitting_thrice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game)))),
    "hitting_four_times":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game))))),
    "doubling":get_return_doubling_down(game)
    })
)

                      standing  hitting_once  hitting_twice  hitting_thrice  hitting_four_times  doubling
blackjack_win_chance  0.000000      0.000000       0.000000        0.000000            0.000000  0.000000
win_chance            0.426381      0.553199       0.295036        0.071599            0.012361  0.553199
push_chance           0.000000      0.062005       0.035860        0.009302            0.001619  0.062005
lose_chance           0.573619      0.384796       0.669104        0.919099            0.986020  0.384796
expected_return      -0.147238      0.168402      -0.374068       -0.847500           -0.973659  0.336805


In [ ]:
game = get_game()

In [ ]:
game = dealer_draws_once(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.076923,0,0,0,2,1,0,23,24,24,24,24,24,24,24,96,24
1,0.076923,0,0,0,3,1,0,24,23,24,24,24,24,24,24,96,24
2,0.076923,0,0,0,4,1,0,24,24,23,24,24,24,24,24,96,24
3,0.076923,0,0,0,5,1,0,24,24,24,23,24,24,24,24,96,24
4,0.076923,0,0,0,6,1,0,24,24,24,24,23,24,24,24,96,24
5,0.076923,0,0,0,7,1,0,24,24,24,24,24,23,24,24,96,24
6,0.076923,0,0,0,8,1,0,24,24,24,24,24,24,23,24,96,24
7,0.076923,0,0,0,9,1,0,24,24,24,24,24,24,24,23,96,24
8,0.307692,0,0,0,10,1,0,24,24,24,24,24,24,24,24,95,24
9,0.076923,0,0,0,11,1,1,24,24,24,24,24,24,24,24,96,23


In [ ]:
game = player_hits_once_where_possible(game)
game = player_hits_once_where_possible(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24
1,0.000440,4,2,0,3,1,0,22,23,24,24,24,24,24,24,96,24
2,0.000440,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24
3,0.000440,4,2,0,5,1,0,22,24,24,23,24,24,24,24,96,24
4,0.000440,4,2,0,6,1,0,22,24,24,24,23,24,24,24,96,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,0.003677,22,2,1,4,1,0,24,24,23,24,24,24,24,24,95,23
541,0.003677,22,2,1,3,1,0,24,23,24,24,24,24,24,24,95,23
548,0.014553,22,2,1,10,1,0,24,24,24,24,24,24,24,24,94,23
543,0.003677,22,2,1,5,1,0,24,24,24,23,24,24,24,24,95,23


In [ ]:
scores = get_return_standing(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       1.427808e-04            17             2            0            17             2            1  24  24  24  24  23  24  23  23  96  23
1       5.711232e-04            12             2            0            17             2            0  24  23  24  24  24  23  24  23  95  24
2       1.368316e-04            12             2            0            17             2            0  24  23  24  24  24  24  23  22  96  24
3       1.427808e-04             9             2            0            17             2            0  24  23  24  24  23  24  23  23  96  24
4       5.711232e-04             9             2            0            17             2            0  24  24  23  23  24  23  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.047489
win_chance              0.339357
push_chance             0.046066
lose_chance             0.567088
expected_return        -0.156497
dtype: float64

In [ ]:
scores = get_return_hitting_once_where_possible(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       6.397321e-05            19             3            0            17             2            0  24  24  24  22  24  23  24  23  95  24
1       1.532692e-05            19             3            0            17             2            0  24  24  24  22  24  24  23  22  96  24
2       1.335093e-04            19             3            0            17             2            0  24  24  24  23  23  23  23  24  95  24
3       3.198661e-05            19             3            0            17             2            0  24  24  24  23  23  24  22  23  96  24
4       5.864211e-05            19             3            0            17             2            0  24  24  24  23  24  21  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.047489
win_chance              0.264836
push_chance             0.049003
lose_chance             0.638671
expected_return        -0.302601
dtype: float64

In [ ]:
#TODO: implement splitting


To Beat:

r = get_best_return(get_game(None, [6,6], [3], update_deck=False))

1:30 min